In [ ]:
# setup
import warnings;
warnings.filterwarnings('ignore'); #tensorflow gives me weird stuff
import numpy as np;
import tensorflow as tf;
from numpy import matmul as mul
from numpy.linalg import norm as norm
from scipy import sparse
from tensorflow.sparse import to_dense
from numba import prange,njit,jit
from numba.typed import List
from datetime import datetime
# tf.enable_eager_execution()

In [ ]:
@njit
def expand(factors, weights, dim_no, cur_idx, all_vals, rank):
    
    if dim_no == len(factors):
#         print(cur_idx)
        value = np.ones(rank)
        for j,w in enumerate(weights):                
            value[j] = w
        for i in range(dim_no):
            ci = cur_idx[i]
            f = factors[i][ci]
            for j,v in enumerate(value):                
                value[j] = v * f[j]
        s = 0.0
        for val in value:
            s += val
        if(s != 0.0):
            t = np.ones(dim_no, dtype=np.int64)
            for k in range(dim_no):
                t[k] = cur_idx[k]
            all_vals.append((t,s)) 
    else:
        cur_fact = factors[dim_no]
        for i in range(len(cur_fact)):
            cur_idx[dim_no] = i
            expand(factors, weights, dim_no + 1, cur_idx, all_vals, rank)

def rebuild(kruskal_tensor, dimensions, rank):
    factors = kruskal_tensor[1]
    weights = kruskal_tensor[0][0]
    cur_idx = np.zeros(len(dimensions), dtype="int64")
    av = [(cur_idx, 0.0)] # list(Tuple(array(int64, 1d, C), float64)))
    
    expand(factors, weights, 0, cur_idx, av, rank)
    
    av = av[1:]
    indexes = [a[0] for a in av]
    vals = [a[1] for a in av]
#     print(indexes)
#     print(vals)
    st = tf.SparseTensor(indices=indexes, values=vals, dense_shape=dimensions)
    return st

In [ ]:
def generate_random_factors(dimensions, rank, d = 0.1):
    factors = [sparse.random(dim,rank,density=d).A for dim in dimensions]
    return factors

@njit(parallel=True)
def l1_erf(factors, cur_idx, all_vals, rank):
    cur_fact = factors[0]
    for i in prange(len(cur_fact)):
        cur_idx[0] = i
        expand_random_factors(factors, 1, cur_idx, all_vals, rank)
        
@njit
def expand_random_factors(factors, dim_no, cur_idx, all_vals, rank):
    #this method just writes to all values, so all values needs to be saved somewhere
    if dim_no == len(factors):
        value = np.ones(rank)
        for i in range(dim_no):
            ci = cur_idx[i]
            f = factors[i][ci]
            for j,v in enumerate(value):                
                value[j] = v * f[j]
        s = 0.0
        for val in value:
            s += val
        v = s * (3.16**dim_no)
        if(v != 0.0):
            t = np.ones(dim_no, dtype=np.int64)
            t *= cur_idx
            all_vals.append((t,v)) 
    else:
        cur_fact = factors[dim_no]
        for i in range(len(cur_fact)):
            cur_idx[dim_no] = i
            expand_random_factors(factors, dim_no + 1, cur_idx, all_vals, rank)
            
def generate_decomposable_sp_tensor(dimensions, rank, d = 0.1):
    nd = len(dimensions)
    factor_d = (d/rank) ** (1/nd)
    
    factors = generate_random_factors(dimensions, rank, factor_d)
    cur_idx = np.zeros(len(dimensions), dtype="int64")
    all_values = [(cur_idx, 0.0)] # list(Tuple(array(int64, 1d, C), float64)))
    expand_random_factors(factors, 0, cur_idx, all_values, rank)
    all_values = all_values[1:]
    indices = [a[0] for a in all_values]
    values = [a[1] for a in all_values]
    shape = dimensions
#     print(indices)
#     print(values)
    st = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
    return st

def generate_random_sp_tensor(dimensions, d = 0.2):
    nd = len(dimensions)
    num_items = min(100000 , (int)(np.prod(dimensions) * d))    
    idxs = set()
    
    for i in range(num_items):
        rand = np.random.rand(nd) #gives us a random index
        index = tuple(np.trunc(np.multiply(rand,dimensions)).astype(int))
        idxs.add(index)
        
    indices = list(idxs)
    values = np.random.rand(len(indices))
    indices.sort()
    st = tf.SparseTensor(indices=indices, values=values, dense_shape=dimensions)
    return st

In [ ]:
def tensor_norm(st):
    return (sum([x**2 for x in st.values.numpy()])**0.5)

def diff(spt1, spt2):
    idx1 = [tuple(s) for s in spt1.indices.numpy()]
    idx2 = [tuple(s) for s in spt2.indices.numpy()]
    val1 = spt1.values.numpy()
    val2 = spt2.values.numpy() 
    s1 = [(idx1[i],val1[i]) for i in range(len(idx1))]
    s2 = [(idx2[i],val2[i]) for i in range(len(idx2))]
    s1.sort()
    s2.sort()
    i1 = 0
    i2 = 0
    l1 = len(s1)
    l2 = len(s2)
    
    sum_sq = 0
    while(i1 < l1 and i2 < l2):
        p1 = s1[i1]
        p2 = s2[i2]
        if p1[0] == p2[0]:
            sum_sq += (p1[1] - p2[1]) ** 2
            i1 += 1
            i2 += 1
        elif p1[0] < p2[0]:
            sum_sq += p1[1] ** 2
            i1 += 1
        else:
            sum_sq += p2[1] ** 2
            i2 += 1
    if(i1 == l1):
        while(i2 < l2):
            p2 = s2[i2]
            sum_sq += p2[1] ** 2
            i2 += 1
    else:
        while(i1 < l1):
            p1 = s1[i1]
            sum_sq += p1[1] ** 2
            i1 += 1
            
    return sum_sq ** 0.5
            
def fit(spt1,spt2):
    return 1 - (diff(spt1,spt2)/tensor_norm(spt1))

def easy_fit(spt1,spt2):
    return 1 - (abs(tensor_norm(spt1)-tensor_norm(spt2))/tensor_norm(spt1))

In [ ]:
def mttkrp2(X, factors, n, rank, dims):    
    output = np.zeros((dims[n],rank))
    indices = X.indices.numpy()
    values = X.values.numpy()
    
    for l in prange(len(values)):
        cur_index = indices[l]
        prod = [values[l]]*rank #makes the value into a row

        for mode,cv in enumerate(cur_index): #does elementwise row multiplications
            if(mode != n):
                prod *= factors[mode][cv]      
        output[cur_index[n]] += prod
    
    return output

In [ ]:
@njit(parallel=True)
def mttkrp(values, indices, factors, n, rank, dims):    
    output = np.zeros((dims[n],rank))

    for l in prange(len(values)):
        cur_index = indices[l]
        prod = [values[l]]*rank #makes the value into a row

        for mode,cv in enumerate(cur_index): #does elementwise row multiplications
            if(mode != n):
                for r in range(rank):
                    prod[r] *= factors[mode][cv][r]
            
        for r in range(rank):
            output[cur_index[n]][r] += prod[r]       
    
    return output

In [ ]:
# CP Decomposition

def cp_als(X, rank, n_iter_max = 50):
    
    dims = X.shape.as_list()
    nd = len(dims)
    factors = [np.random.random((d,rank)) for d in dims]
    weights = np.ones((1,rank))
    
    for iteration in range(n_iter_max): 
#         print(iteration+1 , end="\r")
        for n in range(nd):
            
            #the following block calculates inverse of the hadamard product
            h = mul(weights.T,weights)
            for i,f in enumerate(factors):
                if i != n:
                    h *= mul(f.T,f)
            vinv = np.linalg.pinv(h)
            
            #the following block calculates An by doing MTTKRP and multiplying it by the inverse of the hadamard
            vals = X.values.numpy()
            idxs = X.indices.numpy()
            mk = mttkrp(vals, idxs, factors, n, rank, dims)
#             mk = mttkrp2(X, factors, n, rank, dims)
            wmk = np.multiply(mk, weights[0]) #handling the weights
            An = mul(wmk,vinv)
            
            #the following block normalizes the columns and stored
            weight = norm(An,axis=0)
            b = np.where(weight<1e-12, 1, weight)
            weights[0] *= b
            An /= b
            
            factors[n] = An
            
    return weights, factors

In [ ]:
shape = (20,30,50)
rank = 19
density = 0.2
before = datetime.now()
st = generate_decomposable_sp_tensor(shape, rank, d=density)
after = datetime.now()
print("time: ",  (after-before).seconds, " seconds")
print("actual density: ", st.values.shape[0]/np.prod(shape), " vs given density ", density)
# st = generate_random_sp_tensor(shape, rank)

In [ ]:
before = datetime.now()
cpd = cp_als(st, rank,n_iter_max=50)
after = datetime.now()
print("\nrebuilding\n")
rebuilt = rebuild(cpd,shape,rank)
after2 = datetime.now()
fit_st_rebuilt = fit(st,rebuilt)
after3 = datetime.now()
print("cp_als time: ",  (after-before).seconds, " seconds") # there is about a 20 - 30x speedup between the
print("rebuild time: ",  (after2-after).seconds, " seconds")
print("fit time: ",  (after3-after2).seconds, " seconds")
print("fit ", fit_st_rebuilt)
# print("fit1 ", fit(st,rebuilt)) fit2 is the exact same thing but way faster
# print(cpd)
# print(to_dense(rebuilt))
# print(to_dense(st))
print(st)

In [ ]:
def test(shape, rank, iters=50, density = 0.01, verbose=True):
    t0 = datetime.now()
    st = generate_decomposable_sp_tensor(shape, rank, d=density)
    t1 = datetime.now()
    cpd = cp_als(st, rank,n_iter_max=iters)
    t2 = datetime.now()
    rebuilt = rebuild(cpd,shape,rank)
    t3 = datetime.now()
    fit_st_rebuilt = fit(st,rebuilt)
    t4 = datetime.now()
    
    result_text = '''
    +--------------------------------------------
    | shape: {}
    | rank: {}
    | iterations: {}
    | density: {}
    | actual density: {}
    | number of non-zeros: {}
    |--------------------------------------------
    | time to generate tensor: {} seconds
    | time to perform cp_als:  {} seconds
    | time to rebuild decomp:  {} seconds
    | time to perform fit:     {} seconds
    |--------------------------------------------
    | fit: {}
    +--------------------------------------------
    '''.format(shape, rank, iters, density, 
               (st.values.shape[0]/np.prod(shape)),
               st.values.shape[0],(t1-t0).seconds, 
               (t2-t1).seconds, (t3-t2).seconds, 
               (t4-t3).seconds, fit_st_rebuilt)
    if verbose:
        print(result_text)
    res = {
        "shape":shape,
        "rank":rank,
        "iterations":iters,
        "density":density,
        "actual_density": (st.values.shape[0]/np.prod(shape)),
        "non_zeros":st.values.shape[0],
        "tensor_generate_time":(t1-t0).seconds,
        "cp_als_time":(t2-t1).seconds,
        "redbuild_time":(t3-t2).seconds,
        "fit_time":(t4-t3).seconds,
        "fit":fit_st_rebuilt
    }
    return res

In [ ]:
t = test((400,5000,400), 22, iters=50, density=0.00000007)

In [ ]:
# Brute force way for rebuilding mode-3 tensors

# @njit(parallel=True)
# def rebuild_3(kruskal_tensor, dimensions, rank):
#     if len(dimensions)!= 3:
#         return
#     indices = []
#     values = []
#     factors = kruskal_tensor[1]
#     weights = kruskal_tensor[0][0]
#     fi = factors[0]
#     fj = factors[1]
#     fk = factors[2]
#     for i in range(dimensions[0]):
#         print(i, end="\r")
#         for j in range(dimensions[1]):
#             for k in range(dimensions[2]):
#                 varr = fi[i] * fj[j] * fk[k] * weights
#                 v = 0
#                 for val in varr:
#                     v+= val
#                 if v != 0:
#                     indices.append([i,j,k])
#                     values.append(v)
#     return tf.SparseTensor(indices=indices, values=values, dense_shape=dimensions)


In [ ]:
'''
A robust testing framework should test:
    tensors of varying number of dimensions
    tensors of various sizes
    tensors of varying rank
    tensors of varying sparsity
    tensors that are not perfectly decomposable
    running a variety of different ranks
    running a variety of different max number of iterations
'''